In [1]:
import os
import sys
import gc
from tqdm.notebook import tqdm
sys.path.append('../')
from utiliT.io import read_pickle, dump_pickle
from tqdm.notebook import tqdm
from scripts.instance import *
from scripts.benders import *
import yaml
import json

In [3]:
test_file = 'multipeak_100_150_400.pkl'
instance = read_pickle(f"../data/{test_file}")

In [3]:
MC_cut = multi_cut_bender(instance)
print(MC_cut['obj_val'])

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-20
29390.2533176242


In [4]:
MC_cut = single_cut_benders(instance)
print(MC_cut['obj_val'])

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-20
29390.312382698743


In [ ]:
del MC_cut
gc.collect()

In [2]:
files = os.listdir("../data")
for file in tqdm(files):
    try:
        file_name = file[:-4]
        instance = read_pickle(f"../data/{file}")
        result1 = multi_cut_bender(instance)
        dump_pickle(result1, f"../results/multicut_{file_name}.pkl")
        del result1
        gc.collect()
        result2 = single_cut_benders(instance)
        dump_pickle(result2, f"../results/singlecut_{file_name}.pkl")
    except Exception as e:
        print("--------------")
        print(file)
        print(e)

  0%|          | 0/108 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-20


In [7]:
read_pickle("../results/singlecut_multipeak_200_200_400.pkl")

{'method': 'single-cut',
 'obj_val': 53001.70926933491,
 'n_cuts': 36,
 'n_iterations': 36,
 'avg_mp_solve': 0.009555002053578695,
 'avg_benders_loop_solve': 8.385000381204817,
 'status': 'timelimit',
 'primal_gap': -3.296256155175797,
 'primal_gap_perc': 6.219150666302955e-05,
 'runtime': 303.6321454048157,
 'n1': 200,
 'n2': 200,
 'm1': 400,
 'm2': 400,
 'k': 400,
 'distribution': 'multipeak'}